<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Promptify_GPT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install promptify -q

In [3]:
import json
from promptify.models.nlp.openai_model import OpenAI
from promptify.prompts.nlp.prompter import Prompter
from pprint import pprint

In [4]:
api_key  = "key"

model = OpenAI(api_key)
nlp_prompter = Prompter(model)

sent = "The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection"
print(sent)

The patient is a 93-year-old female with a medical history of chronic right hip pain, osteoporosis, hypertension, depression, and chronic atrial fibrillation admitted for evaluation and management of severe nausea and vomiting and urinary tract infection


### 1: Named Entity Recognition (NER) : Zero Shot Example

In [5]:
result = nlp_prompter.fit('ner.jinja',
                          domain      = 'medical',
                          text_input  = sent, 
                          labels      = None)

pprint(eval(result['text']))

[{'E': '93-year-old', 'T': 'Age'},
 {'E': 'chronic right hip pain', 'T': 'Medical Condition'},
 {'E': 'osteoporosis', 'T': 'Medical Condition'},
 {'E': 'hypertension', 'T': 'Medical Condition'},
 {'E': 'depression', 'T': 'Medical Condition'},
 {'E': 'chronic atrial fibrillation', 'T': 'Medical Condition'},
 {'E': 'severe nausea and vomiting', 'T': 'Symptom'},
 {'E': 'urinary tract infection', 'T': 'Medical Condition'},
 {'Branch': 'Internal Medicine', 'Group': 'Geriatrics'}]


In [7]:
result = nlp_prompter.fit('ner.jinja',
                          domain      = 'mobile',
                          text_input  = "Redmi Note 12 5G Specs: Snapdragon® 4 Gen 1, 33W Fast Charging, 48 MP AI Triple Camera, Super Amoled, Ultra Slim, 120Hz Refresh Rate, India Ready 5G.", 
                          labels      = None)

pprint(eval(result['text']))

[{'E': 'Redmi Note 12 5G', 'T': 'Device'},
 {'E': 'Snapdragon® 4 Gen 1', 'T': 'Processor'},
 {'E': '33W Fast Charging', 'T': 'Charging'},
 {'E': '48 MP AI Triple Camera', 'T': 'Camera'},
 {'E': 'Super Amoled', 'T': 'Display'},
 {'E': 'Ultra Slim', 'T': 'Design'},
 {'E': '120Hz Refresh Rate', 'T': 'Refresh Rate'},
 {'E': 'India Ready 5G', 'T': 'Network'},
 {'Branch': 'Mobile', 'Group': 'Specifications'}]


###2 : Named Entity Recognition (NER) with Custom Tags/Entitie

In [8]:
result = nlp_prompter.fit('ner.jinja',
                          domain      = 'medical',
                          text_input  = sent, 
                          labels      = ["SYMPTOM", "DISEASE"])

pprint(eval(result['text']))

[{'E': 'nausea and vomiting', 'T': 'SYMPTOM'},
 {'E': 'urinary tract infection', 'T': 'SYMPTOM'},
 {'E': 'osteoporosis', 'T': 'DISEASE'},
 {'E': 'hypertension', 'T': 'DISEASE'},
 {'E': 'depression', 'T': 'DISEASE'},
 {'E': 'chronic atrial fibrillation', 'T': 'DISEASE'},
 {'branch': 'Medical', 'group': 'History'}]


###3 : Named Entity Recognition (NER) with One shot Example 

In [10]:
one_shot = "Leptomeningeal metastases (LM) occur in patients with breast cancer (BC) and lung cancer (LC). The cerebrospinal fluid (CSF) tumour microenvironment (TME) of LM patients is not well defined at a single-cell level. We did an analysis based on single-cell RNA sequencing (scRNA-seq) data and four patient-derived CSF samples of idiopathic intracranial hypertension (IIH)"
one_shot = [[one_shot, [{'E': 'DISEASE', 'T': 'Leptomeningeal metastases'}, {'E': 'DISEASE', 'T': 'breast cancer'}, {'E': 'DISEASE', 'T': 'lung cancer'}, {'E': 'BIOMARKER', 'T': 'cerebrospinal fluid'}, {'E': 'DISEASE', 'T': 'tumour microenvironment'}, {'E': 'TEST', 'T': 'single-cell RNA sequencing'}, {'E': 'DISEASE', 'T': 'idiopathic intracranial hypertension'}]]]


result = nlp_prompter.fit('ner.jinja',
                          domain      = 'medical',
                          text_input  = sent,
                          examples    = one_shot,
                          labels      = ["SYMPTOM", "DISEASE"])


pprint(eval(result['text']))

[[{'E': 'SYMPTOM', 'T': 'chronic right hip pain'},
  {'E': 'DISEASE', 'T': 'osteoporosis'},
  {'E': 'DISEASE', 'T': 'hypertension'},
  {'E': 'DISEASE', 'T': 'depression'},
  {'E': 'DISEASE', 'T': 'chronic atrial fibrillation'},
  {'E': 'SYMPTOM', 'T': 'severe nausea and vomiting'},
  {'E': 'DISEASE', 'T': 'urinary tract infection'}]]


### 4: Question-Answering

In [11]:
result = nlp_prompter.fit('qa.jinja',
                                      context=sent,
                                      question="what is the main problem with patient?",
                                      domain="medical",
                                     )


pprint(eval(result['text']))

[{'A': 'Severe nausea and vomiting and urinary tract infection'}]


###5 : Question-Answering Generation

In [29]:
context = "The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well."
result = nlp_prompter.fit('qa_gen.jinja',
                                      text_input=context,
                                      domain="story",
                                      total_questions=3,
                                      max_QA_tokens=10
                                     )


pprint(eval(result['text']))

[{'A': 'Alice found herself falling down a very deep well.',
  'Q': 'What happened when Alice went down the rabbit-hole?'},
 {'A': 'It was very deep.', 'Q': 'How deep was the well Alice fell into?'},
 {'A': 'The rabbit-hole went straight on like a tunnel for some way, and then '
       'dipped suddenly down, so suddenly that Alice had not a moment to think '
       'about stopping herself.',
  'Q': 'How did Alice end up falling down the well?'}]
